# Лабораторная работа 2
## ----------------------------------------------------------------------------------------------------------------------------------
## Задание:
1. Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

#### Подключаю библеотеки

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

#### Загружаю данные и подготавливаю датасет

In [15]:
dataSet = pd.read_csv('cybersecurity_attacks.csv')

label_encoder = LabelEncoder()
dataSet['Attack Type Encoded'] = label_encoder.fit_transform(dataSet['Attack Type'])

non_numeric_columns = dataSet.select_dtypes(exclude=['number']).columns.tolist()

dataSet = dataSet.drop(columns=non_numeric_columns)
dataSet.dropna(inplace=True)
print(dataSet)

       Source Port  Destination Port  Packet Length  Anomaly Scores  \
0            31225             17616            503           28.67   
1            17245             48166           1174           51.50   
2            16811             53600            306           87.42   
3            20018             32534            385           15.79   
4             6131             26646           1462            0.52   
...            ...               ...            ...             ...   
39995        31005              6764           1428           39.28   
39996         2553             28091           1184           27.25   
39997        22505             25152           1043           31.01   
39998        20013              2703            483           97.85   
39999        50137             55575           1175           34.63   

       Attack Type Encoded  
0                        2  
1                        2  
2                        0  
3                        2  
4 

#### Разделяю данные и нормализую

In [16]:
# Разделение данных на обучающую и тестовую выборки
X = dataSet.drop('Attack Type Encoded', axis=1)  # Замените 'target' на имя столбца с целевой переменной
y = dataSet['Attack Type Encoded']  # Замените 'target' на имя столбца с целевой переменной
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Обучение и оценка качества

In [17]:
# Инициализация модели SVM с выбранными гиперпараметрами
# Вы можете изменить эти параметры в соответствии с вашими потребностями
svm_model = SVC(kernel='linear', C=1.0)

# Обучение модели
svm_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = svm_model.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division = 1)

print(f'Точность модели: {accuracy}')
print(report)

# Перекрестная проверка
stratified_kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(svm_model, X_train, y_train, cv=stratified_kfold)

print(f'Средняя точность перекрестной проверки: {cross_val_scores.mean()}')

Точность модели: 0.3295
              precision    recall  f1-score   support

           0       0.33      1.00      0.50      2636
           1       1.00      0.00      0.00      2721
           2       1.00      0.00      0.00      2643

    accuracy                           0.33      8000
   macro avg       0.78      0.33      0.17      8000
weighted avg       0.78      0.33      0.16      8000

Средняя точность перекрестной проверки: 0.3306875
